# 过拟合问题与解决方案

本笔记本展示了过拟合问题的识别和解决方法，包括正则化技术在线性回归和逻辑回归中的应用。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1. 生成过拟合示例数据

In [ ]:
def generate_overfitting_data():
    """
    生成用于演示过拟合问题的数据
    
    返回:
    X: 特征数据 (n_samples, 1)
    y: 目标值 (n_samples, 1)
    """
    n_samples = 20
    # 生成特征值
    X = 6 * np.random.rand(n_samples, 1) - 3
    # 生成目标值，添加较多噪声
    y = 0.5 * X**3 + X**2 + X + 2 + np.random.randn(n_samples, 1) * 5
    return X, y

# 生成数据
X, y = generate_overfitting_data()
print(f"数据形状: X={X.shape}, y={y.shape}")

## 2. 可视化原始数据

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X, y, alpha=0.6)
plt.xlabel('特征 X')
plt.ylabel('目标值 y')
plt.title('过拟合示例数据')
plt.grid(True)
plt.show()

## 3. 多项式回归与过拟合

In [ ]:
def create_polynomial_features(X, degree):
    """
    创建多项式特征
    
    参数:
    X: 原始特征矩阵 (m, 1)
    degree: 多项式次数
    
    返回:
    X_poly: 多项式特征矩阵 (m, degree+1)
    """
    X_poly = np.ones((len(X), 1))
    for d in range(1, degree + 1):
        X_poly = np.hstack((X_poly, np.power(X, d)))
    return X_poly

def linear_regression(X, y):
    """
    使用正规方程求解线性回归
    
    参数:
    X: 特征矩阵 (m, n+1)
    y: 目标值向量 (m, 1)
    
    返回:
    theta: 学习到的参数向量 (n+1, 1)
    """
    return np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

def predict(X, theta):
    """
    使用学习到的参数进行预测
    
    参数:
    X: 特征矩阵 (m, n+1)
    theta: 参数向量 (n+1, 1)
    
    返回:
    predictions: 预测值向量 (m, 1)
    """
    return X.dot(theta)

def compute_mse(y_true, y_pred):
    """
    计算均方误差
    
    参数:
    y_true: 真实目标值 (m, 1)
    y_pred: 预测值 (m, 1)
    
    返回:
    mse: 均方误差值
    """
    return np.mean(np.square(y_true - y_pred))

## 4. 不同阶数多项式的拟合效果

In [ ]:
degrees = [1, 3, 15]
plt.figure(figsize=(18, 6))

for i, degree in enumerate(degrees):
    # 创建多项式特征
    X_poly = create_polynomial_features(X, degree)
    # 训练模型
    theta = linear_regression(X_poly, y)
    
    # 生成测试数据用于绘制曲线
    X_new = np.linspace(-3, 3, 100).reshape(100, 1)
    X_new_poly = create_polynomial_features(X_new, degree)
    y_pred = predict(X_new_poly, theta)
    
    # 计算训练集上的MSE
    y_train_pred = predict(X_poly, theta)
    mse = compute_mse(y, y_train_pred)
    
    # 绘制结果
    plt.subplot(1, 3, i + 1)
    plt.scatter(X, y, alpha=0.6, label='训练数据')
    plt.plot(X_new, y_pred, 'r-', linewidth=2, label=f'{degree}阶多项式')
    plt.xlabel('特征 X')
    plt.ylabel('目标值 y')
    plt.title(f'{degree}阶多项式 (MSE: {mse:.2f})')
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.show()

## 5. 正则化线性回归

In [ ]:
def ridge_regression(X, y, alpha):
    """
    岭回归（正则化线性回归）
    
    参数:
    X: 特征矩阵 (m, n+1)
    y: 目标值向量 (m, 1)
    alpha: 正则化参数
    
    返回:
    theta: 学习到的参数向量 (n+1, 1)
    """
    n_features = X.shape[1]
    # 岭回归的正规方程解
    theta = np.linalg.inv(X.T.dot(X) + alpha * np.eye(n_features)).dot(X.T).dot(y)
    return theta

## 6. 不同正则化参数的效果

In [ ]:
degree = 15  # 高次多项式
alphas = [0, 1, 100]
plt.figure(figsize=(18, 6))

for i, alpha in enumerate(alphas):
    # 创建多项式特征
    X_poly = create_polynomial_features(X, degree)
    # 训练岭回归模型
    theta = ridge_regression(X_poly, y, alpha)
    
    # 生成测试数据用于绘制曲线
    X_new = np.linspace(-3, 3, 100).reshape(100, 1)
    X_new_poly = create_polynomial_features(X_new, degree)
    y_pred = predict(X_new_poly, theta)
    
    # 计算训练集上的MSE
    y_train_pred = predict(X_poly, theta)
    mse = compute_mse(y, y_train_pred)
    
    # 绘制结果
    plt.subplot(1, 3, i + 1)
    plt.scatter(X, y, alpha=0.6, label='训练数据')
    plt.plot(X_new, y_pred, 'r-', linewidth=2, label=f'岭回归 (alpha={alpha})')
    plt.xlabel('特征 X')
    plt.ylabel('目标值 y')
    plt.title(f'岭回归 (alpha={alpha}, MSE: {mse:.2f})')
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.show()

## 7. 正则化逻辑回归

In [ ]:
def sigmoid(z):
    """
    sigmoid函数
    """
    return 1 / (1 + np.exp(-z))

def compute_cost_logistic_regularized(X, y, theta, alpha):
    """
    计算正则化逻辑回归的成本函数
    
    参数:
    X: 特征矩阵 (m, n+1)
    y: 标签向量 (m, 1)
    theta: 参数向量 (n+1, 1)
    alpha: 正则化参数
    
    返回:
    cost: 成本函数值
    """
    m = len(y)
    h = sigmoid(X.dot(theta))
    
    # 计算成本函数（包含正则化项）
    cost = -(1/m) * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h)) + 
           (alpha/(2*m)) * np.sum(np.square(theta[1:]))  # 注意：不正则化偏置项
    
    return cost

def gradient_descent_logistic_regularized(X, y, theta, learning_rate, alpha, n_iterations):
    """
    使用梯度下降算法训练正则化逻辑回归模型
    
    参数:
    X: 特征矩阵 (m, n+1)
    y: 标签向量 (m, 1)
    theta: 初始参数向量 (n+1, 1)
    learning_rate: 学习率
    alpha: 正则化参数
    n_iterations: 迭代次数
    
    返回:
    theta: 学习后的参数向量
    cost_history: 每次迭代的成本函数值
    """
    m = len(y)
    cost_history = np.zeros(n_iterations)
    
    for i in range(n_iterations):
        # 计算预测值
        h = sigmoid(X.dot(theta))
        # 计算梯度（包含正则化项）
        gradient = (1/m) * X.T.dot(h - y)
        # 对非偏置项添加正则化梯度
        gradient[1:] += (alpha/m) * theta[1:]
        # 更新参数
        theta = theta - learning_rate * gradient
        # 记录成本函数值
        cost_history[i] = compute_cost_logistic_regularized(X, y, theta, alpha)
    
    return theta, cost_history

## 8. 正则化解决过拟合总结

In [ ]:
print("过拟合问题总结:")
print("1. 过拟合表现：模型在训练数据上表现很好，但在测试数据上表现较差")
print("2. 过拟合原因：模型复杂度太高，学习了训练数据中的噪声")
print("3. 解决方法：")
print("   - 增加训练数据")
print("   - 减少特征数量")
print("   - 使用正则化技术")
print("   - 使用交叉验证选择模型")

print("\n正则化技术:")
print("1. 岭回归（Ridge Regression）：L2正则化，添加参数平方和的惩罚项")
print("2. LASSO回归：L1正则化，添加参数绝对值和的惩罚项，可用于特征选择")
print("3. 弹性网络（Elastic Net）：结合L1和L2正则化")

print("\n正则化参数alpha:")
print("- alpha=0: 无正则化，可能过拟合")
print("- alpha过大: 正则化太强，可能欠拟合")
print("- 需要通过交叉验证选择合适的alpha值")